# Storytelling with Data! in Altair

by Maisa de Oliveira Fraiz

## Introduction

This project aims to replicate the examples from Cole Nussbaumer's book, "Storytelling with Data - Let's Practice!", using `Python Altair`. Our primary objective is to document the reasoning behind the modifications proposed by the author, while also highlighting the challenges that arise when transitioning from the book's Excel-based approach to programming in a different software environment.

`Altair` was selected for this project due to its declarative syntax, interactivity, grammar of graphics, and compatibility with `Streamlit` and other web formatting tools, while within the user-friendly Python environment. Anticipated challenges include the comparatively smaller documentation and development community of Altair compared to more established libraries like `Matplotlib`, `Seaborn`, or `Plotly`. Furthermore, tasks that might appear straightforward in Excel may require multiple iterations to translate effectively into the language.


## Imports

In [2]:
import pandas as pd
import numpy as np
import altair as alt

## Chapter 5 - Think like a designer

*"Where do you want your audience to look?"* - Cole Nussbaumer

### Exercise 4 - design in style

This exercise revolves around integrating brand design into graphs. While the author in the book focuses on creating a graph with a Coke and Light Coke theme, our approach takes it a step further. We will delve into various color inspirations using the most colorful graph thus far, which is the one from exercise 4.2.

We will separate the exercise into three categories: Accessibility, Branding, Paintings, and Nature.

In [3]:
table = pd.read_excel(r"..\..\Data\4.2 EXERCISE.xlsx", usecols = [1, 2, 3], header = 5, skipfooter = 30)
table['Brands'] = table['Unnamed: 1']
table['Change'] = table['$ Vol % change']

table.drop(columns = ['Unnamed: 1', '$ Vol % change'], inplace = True)
table

,spacing for dot plot,Brands,Change
0,0,Fran's Recipe,-0.14
1,1,Wholesome Goodness,-0.13
2,2,Lifestyle,-0.10
3,3,Coat protection,-0.09
4,4,Diet Lifestyle,-0.08
5,5,Feline Basics,-0.05
6,6,Lifestyle Plus,-0.04
7,7,Feline Freedom,-0.02
8,8,Feline Gold,0.01
9,9,Feline Platinum,0.01


### The Original graph:

In [4]:
decreased_most = table.nsmallest(2, 'Change')
increased_most = table.nlargest(2, 'Change')

brands_decreased = decreased_most['Brands'].tolist()
brands_increased = increased_most['Brands'].tolist()

conditions_decreased = [f'datum.Brands == "{brand}"' for brand in brands_decreased]
condition_decreased = f"({'|'.join(conditions_decreased)})"

conditions_increased = [f'datum.Brands == "{brand}"' for brand in brands_increased]
condition_increased = f"({'|'.join(conditions_increased)})"

chart_gray = alt.Chart(
    table
    ).mark_bar(color = "#c6c6c6", size = 15).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = alt.Axis(grid = False, orient = "top", 
                        labelColor = "#888888", titleColor = '#888888', 
                        titleFontWeight = 'normal', format = "%"),
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None)
    )

chart_oranges_mix = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = alt.Axis(grid = False, orient = "top", 
                        labelColor = "#888888", titleColor = '#888888', 
                        titleFontWeight = 'normal', format = "%"),
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_decreased, alt.value('#ec7c30'), alt.value('#efb284'))
).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )

chart_blue_mix = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = alt.Axis(grid = False, orient = "top", 
                        labelColor = "#888888", titleColor = '#888888', 
                        titleFontWeight = 'normal', format = "%"),
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_increased,
                          alt.value('#4772b8'), alt.value('#91a9d5'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

label1_gray = alt.Chart(table.loc[table['Change'] < 0]).mark_text(align = 'left', color = "#c6c6c6", fontWeight = 700).encode(
    x = alt.value(207),
    y = alt.Y('Brands', sort = None),
    text = alt.Text('Brands')
    )

label2_gray = alt.Chart(table.loc[table['Change'] > 0]).mark_text(align = 'right', color = "#c6c6c6", fontWeight = 700).encode(
    x = alt.value(192),
    y = alt.Y('Brands', sort = None),
    text = alt.Text('Brands')
    )

label_oranges = alt.Chart(table.loc[table['Change'] < 0]).mark_text(align = 'left', fontWeight = 700).encode(
    x = alt.value(207),
    y = alt.Y('Brands', sort = None),
    text = alt.Text('Brands'),
    color = alt.condition(condition_decreased,
                          alt.value('#ec7c30'), 
                          alt.value('#efb284'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )


label_blue = alt.Chart(table.loc[table['Change'] > 0]).mark_text(align = 'right', fontWeight = 700).encode(
    x = alt.value(192),
    y = alt.Y('Brands', sort = None),
    text = alt.Text('Brands'),
    color = alt.condition(condition_increased,
                          alt.value('#4772b8'), 
                          alt.value('#91a9d5'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

title_bw = alt.Chart(
    {"values": [{"text":  ["Cat food brands:"]}]}
    ).mark_text(
        size = 16, align = "left", dx = -200, dy = -270, fontWeight = 'normal', color = 'black'
        ).encode(
            text = "text:N"
            )

title_bw_bold = alt.Chart(
    {"values": [{"text":  [
        'Lifestyle line brands decline'
         ]}]}
).mark_text(size = 16, align = "left", dx = -78, dy = -270, fontWeight = 700, color = 'black').encode(
    text = "text:N"
)

title_bw_bold_2 = alt.Chart(
    {"values": [{"text":  [
        'mixed results in sales year-over-year'
         ]}]}
).mark_text(size = 16, align = "left", dx = -78, dy = -270, fontWeight = 700, color = 'black').encode(
    text = "text:N"
)

subtitle_bw = alt.Chart(
    {"values": [{"text":  [
        "YEAR-OVER-YEAR % CHANGE IN VOLUME ($)"
         ]}]}
).mark_text(size = 11, align = "left", dx = -200, dy = -250, fontWeight = 'normal', color = 'gray').encode(
    text = "text:N"
)

decreased_orange = alt.Chart(
    {"values": [{"text":  [
        'DECREASED'
         ]}]}
).mark_text(size = 11, align = "left", dx = -80, dy = -220, fontWeight = 700, color = '#ec7c30').encode(
    text = "text:N"
)

increased_blue = alt.Chart(
    {"values": [{"text":  [
            'INCREASED'
         ]}]}
).mark_text(size = 11, align = "left", dx = 20, dy = -220, fontWeight = 700, color = '#4772b8').encode(
    text = "text:N"
)

separation =  alt.Chart(
    {"values": [{"text":  [
            '|'
         ]}]}
).mark_text(size = 11, align = "left", dx = 3, dy = -220, fontWeight = 700, color = '#c6c6c6').encode(
    text = "text:N"
)

original = (chart_gray + chart_oranges_mix + 
          chart_blue_mix + label1_gray + 
          label2_gray + label_oranges + label_blue + 
          title_bw + title_bw_bold_2 + subtitle_bw + 
          decreased_orange + increased_blue + separation)

original.properties(width = 400).configure_view(stroke = None)

alt.LayerChart(...)

## Accessibility

How do we assess the accessibility of the graph palette?

#### Colorblind

First, we can start by checking if the original colors are accessible to people with color blindness. The online tool [Coloring for Colorblindness](https://davidmathlogic.com/colorblind/#%23D81B60-%231E88E5-%23FFC107-%23004D40) helps simulate how your selected color palette appears to viewers with protanopia, deuteranopia (both being the inability to tell the difference between red and green), and tritanopia (inability to tell the difference between blue and green, purple and red, and yellow and pink), respectively.

The image below shows that our initial palette can be distinguished by those with these visual deficiency. The website also offers some famous colorblind friendly palette, such as the [Wong palette](https://www.nature.com/articles/nmeth.1618).


![Alt text](\Images\colorblind.png)

#### Black and White

Although the complete inability to distinguish colors is very rare, having a black and white version of your graph can be beneficial in certain situations, such as when intending to print it in a newspaper or an article. By using the Microsoft Photos App, we can preview how our visualization would appear without colors.

![Alt text](\Images\bw.png)

Since the top and bottom colors can not be distinguished easily, we can create a new black and white palette.

In [6]:
chart_gray = alt.Chart(
    table
    ).mark_bar(color = "#666666", size = 15).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = alt.Axis(grid = False, orient = "top", 
                        labelColor = "#888888", titleColor = '#888888', 
                        titleFontWeight = 'normal', format = "%"),
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None)
    )

chart_light_gray_mix = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = alt.Axis(grid = False, orient = "top", 
                        labelColor = "#888888", titleColor = '#888888', 
                        titleFontWeight = 'normal', format = "%"),
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_decreased, alt.value('#bbbbbb'), alt.value('#999999'))
).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )

chart_black_mix = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = alt.Axis(grid = False, orient = "top", 
                        labelColor = "#888888", titleColor = '#888888', 
                        titleFontWeight = 'normal', format = "%"),
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_increased,
                          alt.value('black'), alt.value('#333333'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

label1_gray = alt.Chart(table.loc[table['Change'] < 0]).mark_text(align = 'left', color = "#666666", fontWeight = 700).encode(
    x = alt.value(207),
    y = alt.Y('Brands', sort = None),
    text = alt.Text('Brands')
    )

label2_gray = alt.Chart(table.loc[table['Change'] > 0]).mark_text(align = 'right', color = "#666666", fontWeight = 700).encode(
    x = alt.value(192),
    y = alt.Y('Brands', sort = None),
    text = alt.Text('Brands')
    )

label_light_gray = alt.Chart(table.loc[table['Change'] < 0]).mark_text(align = 'left', fontWeight = 700).encode(
    x = alt.value(207),
    y = alt.Y('Brands', sort = None),
    text = alt.Text('Brands'),
    color = alt.condition(condition_decreased,
                          alt.value('#bbbbbb'), 
                          alt.value('#999999'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )


label_black = alt.Chart(table.loc[table['Change'] > 0]).mark_text(align = 'right', fontWeight = 700).encode(
    x = alt.value(192),
    y = alt.Y('Brands', sort = None),
    text = alt.Text('Brands'),
    color = alt.condition(condition_increased,
                          alt.value('black'), 
                          alt.value('#333333'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

title_bw = alt.Chart(
    {"values": [{"text":  ["Cat food brands:"]}]}
    ).mark_text(
        size = 16, align = "left", dx = -200, dy = -270, fontWeight = 'normal', color = 'black'
        ).encode(
            text = "text:N"
            )

decreased_light_gray = alt.Chart(
    {"values": [{"text":  [
        'DECREASED'
         ]}]}
).mark_text(size = 11, align = "left", dx = -80, dy = -220, fontWeight = 700, color = '#999999').encode(
    text = "text:N"
)

increased_black = alt.Chart(
    {"values": [{"text":  [
            'INCREASED'
         ]}]}
).mark_text(size = 11, align = "left", dx = 20, dy = -220, fontWeight = 700, color = 'black').encode(
    text = "text:N"
)

black_and_white = (chart_gray + chart_light_gray_mix + 
          chart_black_mix + label1_gray + 
          label2_gray + label_light_gray + label_black + 
          title_bw + title_bw_bold_2 + subtitle_bw + 
          decreased_light_gray + increased_black + separation)

black_and_white.properties(width = 400).configure_view(stroke = None)

alt.LayerChart(...)

## Branding

Branding is a really important aspect of marketing. Most companies have set color palettes, logos and design. Tools such as [Image Color Picker](https://imagecolorpicker.com/) and [Adobe Color](https://color.adobe.com/pt/create/image) can help you extract and implement those official palettes into themed data visualizations.

#### Cookie Clicker

[Cookie Clicker](https://orteil.dashnet.org/cookieclicker/) is an idle game created by the French programmer Orteil, and it has been open in a separate tab baking and accumulating cookies throughout the entirety of this project. Hence, it only seems fitting to pay homage with a dedicated color palette!

In [7]:
chart_middle = alt.Chart(
    table
    ).mark_bar(color = "#ee8241", size = 15).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None)
    )

chart_up = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_decreased, alt.value('#4a251d'), alt.value('#64433a'))
).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )

chart_down = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_increased,
                          alt.value('#81532d'), alt.value('#c0a681'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

cookie_clicker = (chart_middle + chart_down + chart_up)

cookie_clicker.properties(width = 400).configure_view(stroke = None)

alt.LayerChart(...)

![Alt text](\Images\cookieclicker.jpg)

#### Google Maps

One of the most distinctive brand designs is the vibrant Google palette, widely employed in various apps associated with the company. Inspired by the Google Maps logo, which features precisely five colors, we will demonstrate how our graph looks when incorporating this design.

In [8]:
chart_middle = alt.Chart(
    table
    ).mark_bar(color = "#34A852", size = 15).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None)
    )

chart_up = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_decreased, alt.value('#EA4335'), alt.value('#FABB04'))
).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )

chart_down = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_increased,
                          alt.value('#1A73E8'), alt.value('#4285F4'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

google = (chart_middle + chart_down + chart_up)

google.properties(width = 400).configure_view(stroke = None)

alt.LayerChart(...)

![Alt text](\Images\googlemapslogo.png)

## Paintings

Artist spend their lives dedicated to the study of colors and how we as humans perceive them. We can draw inspiration from their work when searching to evoke a specific emotion through our visualizations.

#### Starry Night by Vincent Van Gogh

In [9]:
chart_middle = alt.Chart(
    table
    ).mark_bar(color = "#7B95A6", size = 15).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None)
    )

chart_up = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_decreased, alt.value('#A65D05'), alt.value('#D9B13B'))
).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )

chart_down = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_increased,
                          alt.value('#0B1E38'), alt.value('#304F8C'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

starry_night = (chart_middle + chart_down + chart_up)

starry_night.properties(width = 400).configure_view(stroke = None)

alt.LayerChart(...)

![Alt text](\Images\starry_night.jpg)

#### A Cuca by Tarsila do Amaral

In [10]:
chart_middle = alt.Chart(
    table
    ).mark_bar(color = "#034001", size = 15).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None)
    )

chart_up = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_decreased, alt.value('#0339A6'), alt.value('#067302'))
).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )

chart_down = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_increased,
                          alt.value('#F25C05'), alt.value('#F29F05'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

cuca = (chart_middle + chart_down + chart_up)

cuca.properties(width = 400).configure_view(stroke = None)

alt.LayerChart(...)

![Alt text](\Images\cuca.webp)

#### The Great Wave Off Kanagawa by Hokusai

In [11]:
chart_middle = alt.Chart(
    table
    ).mark_bar(color = "#8FBABF", size = 15).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None)
    )

chart_up = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_decreased, alt.value('#E0C6A3'), alt.value('#D9B779'))
).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )

chart_down = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_increased,
                          alt.value('#010326'), alt.value('#010B40'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

wave = (chart_middle + chart_down + chart_up)

wave.properties(width = 400).configure_view(stroke = None)

alt.LayerChart(...)

![Alt text](\Images\great_wave.jpg)

## Nature

#### Sunset

This picture was taken by Sergio Mena Ferraira and can be found [here](https://unsplash.com/pt-br/fotografias/ondas-do-oceano-batendo-na-costa-durante-o-por-do-sol-N5sC8zZUgy4).

In [12]:
chart_middle = alt.Chart(
    table
    ).mark_bar(color = "#FF9200", size = 15).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None)
    )

chart_up = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_decreased, alt.value('#AA4650'), alt.value('#FD4044'))
).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )

chart_down = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_increased,
                          alt.value('#FA4210'), alt.value('#FD7E37'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

sunset = (chart_middle + chart_down + chart_up)

sunset.properties(width = 400).configure_view(stroke = None)

alt.LayerChart(...)

![Alt text](\Images\sunset.webp)

#### Forest

This picture was taken 

In [13]:
chart_middle = alt.Chart(
    table
    ).mark_bar(color = "#5C7346", size = 15).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None)
    )

chart_up = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_decreased, alt.value('#ABD9A9'), alt.value('#83A66A'))
).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ["Fran's Recipe", 'Wholesome Goodness',
                                                     'Lifestyle', 'Coat protection', 'Diet Lifestyle'])
    )

chart_down = alt.Chart(table).mark_bar( 
        size = 15
        ).encode(
    x = alt.X(
        "Change", 
        scale = alt.Scale(domain = [-0.20, 0.20]), 
        axis = None,
        title = None
        ),
    y = alt.Y("Brands", sort = None, axis = None),
    color = alt.condition(condition_increased,
                          alt.value('#140F09'), alt.value('#3B401B'))
    ).transform_filter(
    alt.FieldOneOfPredicate(field='Brands', oneOf = ['Feline Focus', 'Feline Grain Free', 'Feline Silver',
                                                    'Nutri Balance', 'Farm Fresh Basics'])
    )

forest = (chart_middle + chart_down + chart_up)

forest.properties(width = 400).configure_view(stroke = None)

alt.LayerChart(...)

![Alt text](\Images\forest.jpg)